# Survey Monkey Data Transformation

### Data Sources:
- https://drive.google.com/drive/folders/1ZQJ37FcW4W_tYbz5nEkYX4Y1fqZzfdiF

### Purposes:
- Transform 'raw' data output from SurveyMonkey file into a more easily readable document.
- Optimise data cleaning and manipulation methodologies.
- Familiarise processes to more diverse data types and transformation functions.

### Import Pandas and OS

In [1]:
# import data analysis packages
import pandas as pd
import os

### Load Data

In [2]:
# select folder path strig to variable cwd (current working directory)
pwd = os.getcwd()

# import .xlsx data file dataset (using pwd directory)
data = pd.read_excel(pwd + "\Data Output1_Edited.xlsx",
                     sheet_name="edited_data")

# create a copy of the original data
data_mod = data.copy() 


# import survey questions sparately
questions = pd.read_excel(pwd + "\Data Output1_Edited.xlsx",
                     sheet_name="question")

# create copy of questions data
quest = questions.copy()

### 1) Clean up data frame and unpivot into tall format

In [14]:
# inspect data and remove unwanted columns
print(data_mod.columns) 
columns_to_drop = ['Start Date', 'End Date', 'Email Address', 'First Name', 'Last Name', 'Custom Data 1']

data_modified = data_mod.drop(columns = columns_to_drop)


# select id_vars (to remain constant) and vars_columns (to unpivot) columns 
id_columns = ['Respondent ID', 'Identify which division you work in. - ',       # Columns up to 'Question 1' are 'demographic info' columns
       'Identify which division you work in. - Other (please specify)',         # Keep those 8 unpivoted as 'id_columns'
       'Which of the following best describes your position level? - Response', 
       'Which generation are you apart of? - Response',
       'Please select the gender in which you identify. - Response',
       'Which duration range best aligns with your tenure at your company? - Response',
       'Which of the following best describes your employment type? - Response']

vars_columns = list(data_mod.columns[8:])

# unpivot data into tall format
data_mod1 = data_modified.melt(id_vars=id_columns,
                          value_vars=vars_columns,
                          var_name="Question", value_name="Answer")

Index(['Respondent ID', 'Start Date', 'End Date', 'Email Address',
       'First Name', 'Last Name', 'Custom Data 1',
       'Identify which division you work in. - ',
       'Identify which division you work in. - Other (please specify)',
       'Which of the following best describes your position level? - Response',
       'Which generation are you apart of? - Response',
       'Please select the gender in which you identify. - Response',
       'Which duration range best aligns with your tenure at your company? - Response',
       'Which of the following best describes your employment type? - Response',
       'Question 1 - Response', 'Question 2 - Response',
       'Question 3 - Open-Ended Response', 'Question 4 - Response',
       'Question 4 - Other (please specify)', 'Question 5 - Response 1',
       'Question 5 - Response 2', 'Question 5 - Response 3',
       'Question 5 - Response 4', 'Question 5 - Response 5',
       'Question 5 - Response 6', 'Question 6 - Response 1',
     

### 2) Clean up Questions data frame & join questions

In [13]:
# select question columns of interest
quest_final = pd.DataFrame(quest.loc[:, ["Question", "Question + SubQuestion"]])

# drop null values
quest_final.dropna(inplace=True)  

# join quest_final to data_mod1 to add 'Question + SubQuestion' column
data_merged = pd.merge(left=data_mod1,
                       right=quest_final,
                       how="left",
                       left_on="Question",
                       right_on="Question + SubQuestion")

# double-check output columns
print(data_merged.columns)

print("Original Data", len(data_mod1))                                      
print("Merged Data", len(data_merged))     


Index(['Respondent ID', 'Identify which division you work in. - ',
       'Identify which division you work in. - Other (please specify)',
       'Which of the following best describes your position level? - Response',
       'Which generation are you apart of? - Response',
       'Please select the gender in which you identify. - Response',
       'Which duration range best aligns with your tenure at your company? - Response',
       'Which of the following best describes your employment type? - Response',
       'Question_x', 'Answer', 'Question_y', 'Question + SubQuestion'],
      dtype='object')
Original Data 17028
Merged Data 17028


### 3) Calculate number of respondents per question & join results to data

In [18]:
# remove null rows in Answer column
respondents = data_merged[data_merged["Answer"].notna()]
print(respondents.shape)                                            # slimmed data down from 17,000 to now just 9,664 rows of data by removing null values

# calculate number of respondants per question
respondents_calc = respondents.groupby("Question_y")["Respondent ID"].nunique().reset_index()  

# rename 'Respondent ID' column 
respondents_calc.rename(columns={"Respondent ID":"Respondent Count"}, 
                   inplace=True)

# join calculated column to data
data_merged_2 = pd.merge(left=data_merged,
                       right=respondents_calc,
                       how="left",
                       left_on="Question_y",
                       right_on="Question_y")

# Double check no data has been lost
print("Original Data", len(data_merged))
print("Merged Data", len(data_merged_2))

(9664, 12)
Original Data 17028
Merged Data 17028


### 4) Calculate number of respondents giving the same answer per question

In [7]:
# use 'raw' data from above for calculations
same_answer = data_merged

# calculate unique number of respondants providing same answer to each question & subquestion combo
same_answer = same_answer.groupby(["Question + SubQuestion", "Answer"])["Respondent ID"].nunique().reset_index() 

# rename 'Respondent ID' column
same_answer.rename(columns={"Respondent ID":"Same Answer Count"},   
                   inplace=True)

# join calculated column to data
data_merged_3 = pd.merge(left=data_merged_2,
                       right=same_answer,
                       how="left",
                       left_on=["Question + SubQuestion", "Answer"],
                       right_on=["Question + SubQuestion", "Answer"])

# double-check output columns
print(data_merged_3.columns)

print("Original Data", len(data_merged))
print("Merged Data", len(data_merged_3))

Index(['Respondent ID', 'Identify which division you work in. - ',
       'Identify which division you work in. - Other (please specify)',
       'Which of the following best describes your position level? - Response',
       'Which generation are you apart of? - Response',
       'Please select the gender in which you identify. - Response',
       'Which duration range best aligns with your tenure at your company? - Response',
       'Which of the following best describes your employment type? - Response',
       'Question_x', 'Answer', 'Question_y', 'Question + SubQuestion',
       'Respondent Count', 'Same Answer Count'],
      dtype='object')
Original Data 17028
Merged Data 17028


### 5) Clean up merged data to generate final output file

In [22]:
# replace null values with 0
data_merged_3["Same Answer Count"].fillna(0, inplace=True)

# check if any null values left
data_merged_3["Same Answer Count"].isna().sum()


# find columns to rename
print(data_merged_3.columns)

# make a copy of data
output = data_merged_3.copy()               

# rename demographic questions columns with shorter names
output.rename(columns={'Identify which division you work in. - ':'Division Primary',
       'Identify which division you work in. - Other (please specify)':'Division Secondary',
       'Which of the following best describes your position level? - Response':'Position',
       'Which generation are you apart of? - Response': 'Generation',
       'Please select the gender in which you identify. - Response':'Gender',
       'Which duration range best aligns with your tenure at your company? - Response':'Tenure',
       'Which of the following best describes your employment type? - Response':'Employment Type'},
       inplace=True)

print(output.columns)

# drop columns 'Question_x','Answer', 'Question_y'
output_clean = output.drop(columns=["Question_x", "Answer", "Question_y"]) 

# visualise final processed/output file
output_clean.head(15)

Index(['Respondent ID', 'Identify which division you work in. - ',
       'Identify which division you work in. - Other (please specify)',
       'Which of the following best describes your position level? - Response',
       'Which generation are you apart of? - Response',
       'Please select the gender in which you identify. - Response',
       'Which duration range best aligns with your tenure at your company? - Response',
       'Which of the following best describes your employment type? - Response',
       'Question_x', 'Answer', 'Question_y', 'Question + SubQuestion',
       'Respondent Count', 'Same Answer Count'],
      dtype='object')
Index(['Respondent ID', 'Division Primary', 'Division Secondary', 'Position',
       'Generation', 'Gender', 'Tenure', 'Employment Type', 'Question_x',
       'Answer', 'Question_y', 'Question + SubQuestion', 'Respondent Count',
       'Same Answer Count'],
      dtype='object')


,Respondent ID,Division Primary,Division Secondary,Position,Generation,Gender,Tenure,Employment Type,Question + SubQuestion,Respondent Count,Same Answer Count
0,5379192392,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1 - Response,119,0.0
1,2658722536,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1 - Response,119,17.0
2,4044163394,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,119,22.0
3,5535865599,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 - Response,119,14.0
4,3356802928,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 - Response,119,0.0
5,3399511781,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,119,16.0
6,9860597462,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 1 - Response,119,16.0
7,1494621946,Infrastructure,NaN,Chief / Executive,Baby Boomer (born between 1946-1964),Male,3-5 years,Full time Employee,Question 1 - Response,119,0.0
8,8646387163,Information Technology,NaN,Manager,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1 - Response,119,13.0
9,2455127238,Information Technology,NaN,Staff,Millennial (born between 1981-2000),Male,0-2 years,Full time Employee,Question 1 - Response,119,17.0


## Export final data file

In [9]:
# export final output file
writer = pd.ExcelWriter("Final_Output.xlsx", 
                        engine="xlsxwriter")
output_clean.to_excel(writer, sheet_name="data",
                      index=False) 
writer.close()